# 데이터시각화특강 기말평가 답안 (문제2)
## 202055364 황성윤

## 2. COVID19 시도별/월별 확산과정 시각화 (40점)

아래는 COVID19 확진자수를 지역별로 매일 기록한 자료이다.

https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/covid19_20211202.csv


일별로 기록된 COVID19 확진자수를 월별로 통합한 뒤 2021-01 ~ 2021-10 기간의 발생률을 계산하여 시각화하라. 시각화는 `plotly`의 `choropleth_mapbox`를 이용하며 시간의 추이를 표현하기 위해 `animation_frame` 옵션을 사용한다. 시각화 예시는 아래와 같다.

In [1]:
import folium 
import pandas as pd 
import numpy as np
import json 
import requests 
import plotly.express as px
from IPython.display import HTML

In [2]:
covid_example=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/covid19_20211202.csv')
covid_example

,일자,계(명),서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주,검역
0,누적(명),"457,612","158,774","16,555","19,114","25,299","6,353","8,809","5,675","1,588","136,546","8,889","8,942","13,174","6,453","4,498","11,471","15,236","3,762","6,474"
1,2020-01-20,1,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,2020-01-21,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,2020-01-22,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,2020-01-23,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,2021-11-28,"3,925","1,673",148,106,278,52,53,4,5,"1,090",63,25,121,45,25,103,89,35,10
680,2021-11-29,"3,308","1,393",144,88,233,61,43,2,15,910,56,33,52,49,28,68,86,44,3
681,2021-11-30,"3,032","1,186",79,78,192,52,43,3,22,909,84,59,81,50,36,68,60,22,8
682,2021-12-01,"5,123","2,222",143,86,326,29,88,17,20,"1,582",105,48,96,50,40,97,127,27,20


In [3]:
covid_example.columns

Index(['일자', '계(명)', '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기',
       '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '검역'],
      dtype='object')

In [4]:
covid_ver1=covid_example.iloc[348:652,:].replace('-','0').\
iloc[:,map(lambda x: x != '계(명)' and x != '검역', covid_example.columns)].\
set_index('일자').applymap(lambda x: float(x.replace(',',''))).reset_index().\
melt(id_vars='일자')
covid_ver1

,일자,variable,value
0,2021-01-01,서울,357.0
1,2021-01-02,서울,250.0
2,2021-01-03,서울,198.0
3,2021-01-04,서울,329.0
4,2021-01-05,서울,199.0
...,...,...,...
5163,2021-10-27,제주,7.0
5164,2021-10-28,제주,4.0
5165,2021-10-29,제주,4.0
5166,2021-10-30,제주,9.0


In [5]:
covid_ver1['ym']=covid_ver1['일자']
for i in range(len(covid_ver1)):
    covid_ver1['ym'][i]=covid_ver1['일자'][i][:7]
    
covid_ver1    

C:\Users\HSY\AppData\Local\Temp/ipykernel_28892/1260919139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_ver1['ym'][i]=covid_ver1['일자'][i][:7]


,일자,variable,value,ym
0,2021-01-01,서울,357.0,2021-01
1,2021-01-02,서울,250.0,2021-01
2,2021-01-03,서울,198.0,2021-01
3,2021-01-04,서울,329.0,2021-01
4,2021-01-05,서울,199.0,2021-01
...,...,...,...,...
5163,2021-10-27,제주,7.0,2021-10
5164,2021-10-28,제주,4.0,2021-10
5165,2021-10-29,제주,4.0,2021-10
5166,2021-10-30,제주,9.0,2021-10


In [6]:
covid_ver1=covid_ver1.groupby(['ym','variable']).agg(np.sum).stack().reset_index().\
iloc[:,[0,1,3]].rename(columns={'variable':'prov',0:'covid19'})
covid_ver1

,ym,prov,covid19
0,2021-01,강원,488.0
1,2021-01,경기,5353.0
2,2021-01,경남,686.0
3,2021-01,경북,593.0
4,2021-01,광주,702.0
...,...,...,...
165,2021-10,전남,459.0
166,2021-10,전북,672.0
167,2021-10,제주,225.0
168,2021-10,충남,1467.0


In [7]:
df=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-prov.csv')\
.sort_values('행정구역(시군구)별').reset_index().iloc[:,1:].rename(columns={'총인구수 (명)':'total'})
df['prov']=covid_ver1['prov']
df

,행정구역(시군구)별,total,prov
0,강원도,1537717,강원
1,경기도,13549577,경기
2,경상남도,3318161,경남
3,경상북도,2627925,경북
4,광주광역시,1442454,광주
5,대구광역시,2390721,대구
6,대전광역시,1454228,대전
7,부산광역시,3356311,부산
8,서울특별시,9532428,서울
9,세종특별자치시,368276,세종


In [8]:
covid_ver2=pd.merge(df,covid_ver1).iloc[:,[0,1,3,4]].rename(columns={'행정구역(시군구)별':'prov'})
covid_ver2['prop']=covid_ver2['covid19']/covid_ver2['total']
covid_ver2=covid_ver2.sort_values(['prov','ym'])
covid_ver2

,prov,total,ym,covid19,prop
0,강원도,1537717,2021-01,488.0,0.000317
1,강원도,1537717,2021-02,169.0,0.000110
2,강원도,1537717,2021-03,466.0,0.000303
3,강원도,1537717,2021-04,354.0,0.000230
4,강원도,1537717,2021-05,501.0,0.000326
...,...,...,...,...,...
165,충청북도,1596948,2021-06,363.0,0.000227
166,충청북도,1596948,2021-07,544.0,0.000341
167,충청북도,1596948,2021-08,1302.0,0.000815
168,충청북도,1596948,2021-09,1192.0,0.000746


In [9]:
json_korea_prov='https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json'
json_korea = json.loads(requests.get(json_korea_prov).text)
[json_korea['features'][i]['properties']['name'] for i in range(17)]

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [10]:
fig = px.choropleth_mapbox(covid_ver2,  ### 데이터프레임 
                           geojson=json_korea, ### json파일 
                           color="prop", ### df에서 코로플레스의 단계를 표시 
                           locations="prov", ### df에 존재하는 연결변수
                           animation_frame="ym",
                           featureidkey="properties.name", ### json에 존재하는 연결매개체
                           center={"lat": 36, "lon": 128}, 
                           mapbox_style="carto-positron",
                           range_color=(0, covid_ver2.prop.max()),
                           height=1200,
                           zoom=6.5)
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show(config=dict({'scrollZoom':False}))
#_html=fig.to_html(include_mathjax=False, config=dict({'scrollZoom':False}))
#HTML(_html)